In [1]:
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

In [2]:
asignacion_path = 'bases/asignacion/base_asignacion_20240904_ TRABAJADA.xlsx'
df_asignacion = pd.read_excel(asignacion_path)

In [3]:
print(df_asignacion.shape)
print(df_asignacion.dtypes)
df_asignacion.head(5)

(240733, 42)
codigo                        int64
contrato                      int64
cont_18                       int64
nombre                       object
clave                        object
producto                     object
DiasVenc                      int64
Moneda                       object
Capital                     float64
liqtotal                    float64
agencia                      object
Camp                        float64
Gestor                       object
Tipdoc                       object
Numdoc                       object
oficina                      object
territorio                   object
Tramo                        object
codoficina                    int64
capital_soles               float64
CAMPAÑA                     float64
tipo_fondo                   object
foco                         object
flag_honrado                  int64
ambito                       object
TIPO_CARTERA                 object
desc_tramo                   object
MOTIVO_INHIBICI

,codigo,contrato,cont_18,nombre,clave,producto,DiasVenc,Moneda,Capital,liqtotal,...,ID_VTA.,ID_GARANT,ID_CONDONACION MORA_TEMP,ID_FRACCIONADO,ID_KOBSA_P.CANCELACION,ID_REPORTE_CANC,ID_CARTERA,ID_FLAG,OBSERVACION,AGENCIA CORRECTA
0,2569,110107065001274140,1101075001274140,GLADYS MERELYN ABANTO GONZALES,KSTBC,TARJETA,2767,PEN,53.99,524.36,...,-,-,-,-,-,-,Stock,-,OK,ASESCOM RJ
1,14583,110123765001222660,1101235001222660,ADMINISTRACION DE EMPRESAS LIBRERIA EDITORIAL SAC,BC,TARJETAS PYMES,1512,PEN,37236.78,215218.76,...,-,-,-,-,-,-,Stock,-,OK,SIN AGENCIA
2,14583,110123755001218442,1101235001218442,ADMINISTRACION DE EMPRESAS LIBRERIA EDITORIAL SAC,BC,TARJETAS PYMES,1420,PEN,9955.48,43312.83,...,-,-,-,-,-,-,Stock,-,OK,SIN AGENCIA
3,16896,110130265001000046,1101305001000046,PABLO AGARIE MIYASATO,KSTBC,TARJETA,2344,PEN,35.98,162.04,...,-,-,-,-,-,-,Stock,-,OK,ASESCOM RJ
4,17507,110809188100538643,1108098100538643,AGENCIA DE ADUANA YVAPORES LABARTHE SA,BC,LEASING,1954,USD,102860.52,1729982.12,...,-,-,-,-,-,-,Stock,-,OK,SIN AGENCIA


In [31]:
df_asignacion_test = df_asignacion.copy()
df_asignacion_test.columns

Index(['codigo', 'contrato', 'cont_18', 'nombre', 'clave', 'producto',
       'DiasVenc', 'Moneda', 'Capital', 'liqtotal', 'agencia', 'Camp',
       'Gestor', 'Tipdoc', 'Numdoc', 'oficina', 'territorio', 'Tramo',
       'codoficina', 'capital_soles', 'CAMPAÑA', 'tipo_fondo', 'foco',
       'flag_honrado', 'ambito', 'TIPO_CARTERA', 'desc_tramo',
       'MOTIVO_INHIBICION', 'Hibrido', 'ESTUDIO ABOGADO', 'FECHA_MORA',
       'ID_CASTIGO', 'ID_VTA.', 'ID_GARANT', 'ID_CONDONACION MORA_TEMP',
       'ID_FRACCIONADO', 'ID_KOBSA_P.CANCELACION', 'ID_REPORTE_CANC',
       'ID_CARTERA', 'ID_FLAG', 'OBSERVACION', 'AGENCIA CORRECTA'],
      dtype='object')

In [32]:
cols_required = ['codigo', 'cont_18', 'clave', 'nombre', 'Moneda', 'Capital', 'Camp', 'Gestor', 'CAMPAÑA', 'TIPO_CARTERA', 'AGENCIA CORRECTA', 'tipo_fondo', 'oficina', 'territorio', 'ID_VTA.']

df_asignacion_test = df_asignacion_test[cols_required]
df_asignacion_test.head(1)

,codigo,cont_18,clave,nombre,Moneda,Capital,Camp,Gestor,CAMPAÑA,TIPO_CARTERA,AGENCIA CORRECTA,tipo_fondo,oficina,territorio,ID_VTA.
0,2569,1101075001274140,KSTBC,GLADYS MERELYN ABANTO GONZALES,PEN,53.99,0.75,Alejandra Recavarren,13.4975,UNSECURED,ASESCOM RJ,NaN,OF.FAUCETT,DIVISION LIMA OESTE,-


In [33]:
df_asignacion_test.rename(columns={
    'codigo': 'CODIGO CENTRAL', 
    'cont_18': 'N° CONTRATO', 
    'clave': 'CARTERA', 
    'nombre': 'NOMBRE DEL CLIENTE', 
    'Moneda': 'MONEDA', 
    'Capital': 'DEUDA CAPITAL', 
    'Camp': '% CONDONACIÓN', 
    'Gestor': 'GESTOR', 
    'CAMPAÑA': 'MONTO A PAGAR', 
    'TIPO_CARTERA': 'TIPO CARTERA', 
    'AGENCIA CORRECTA': 'AGENCIA DE COBRANZA', 
    'tipo_fondo': 'TIPO DE FONDO', 
    'oficina': 'OFICINA', 
    'territorio': 'TERRITORIO', 
    'ID_VTA.': 'ID_VENTA'
}, inplace=True)
df_asignacion_test.head(1)

,CODIGO CENTRAL,N° CONTRATO,CARTERA,NOMBRE DEL CLIENTE,MONEDA,DEUDA CAPITAL,% CONDONACIÓN,GESTOR,MONTO A PAGAR,TIPO CARTERA,AGENCIA DE COBRANZA,TIPO DE FONDO,OFICINA,TERRITORIO,ID_VENTA
0,2569,1101075001274140,KSTBC,GLADYS MERELYN ABANTO GONZALES,PEN,53.99,0.75,Alejandra Recavarren,13.4975,UNSECURED,ASESCOM RJ,NaN,OF.FAUCETT,DIVISION LIMA OESTE,-


In [7]:
#df_exj = df_asignacion_test[df_asignacion_test['CARTERA'] == 'EXJ']
#df_exj['CARTERA'].value_counts()

In [8]:
#gestores_permitidos = ['Richard Gonzales Chavez', 'Cynthia Rosales', 'Cynthia Llanos Rosales']
#df_bc_kstbc = df_asignacion_test[(df_asignacion_test['CARTERA'].isin(['BC', 'KSTBC'])) & (df_asignacion_test['GESTOR'].isin(gestores_permitidos))]
#df_bc_kstbc['CARTERA'].value_counts()

In [9]:
#df_bc_kstbc['GESTOR'].value_counts()

In [10]:
#df_filtrado = pd.concat([df_exj, df_bc_kstbc])
#df_filtrado.reset_index(drop=True, inplace=True)
#print(df_filtrado.shape)
#df_filtrado.head(5)

In [34]:
df_asignacion_test = df_asignacion_test[df_asignacion_test['TIPO CARTERA'] == 'UNSECURED']
df_asignacion_test = df_asignacion_test[df_asignacion_test['ID_VENTA'].isin(['-'])]
df_asignacion_test = df_asignacion_test[~df_asignacion_test['AGENCIA DE COBRANZA'].isin(['SIN AGENCIA'])]
df_asignacion_test.columns

Index(['CODIGO CENTRAL', 'N° CONTRATO', 'CARTERA', 'NOMBRE DEL CLIENTE',
       'MONEDA', 'DEUDA CAPITAL', '% CONDONACIÓN', 'GESTOR', 'MONTO A PAGAR',
       'TIPO CARTERA', 'AGENCIA DE COBRANZA', 'TIPO DE FONDO', 'OFICINA',
       'TERRITORIO', 'ID_VENTA'],
      dtype='object')

In [35]:
df_asignacion_test['CODIGO CENTRAL'] = df_asignacion_test['CODIGO CENTRAL'].astype('Int64').astype(str).str.zfill(8)
df_asignacion_test['N° CONTRATO'] = df_asignacion_test['N° CONTRATO'].apply(lambda x: str(int(x)).zfill(18) if pd.notna(x) else x)
df_asignacion_test['MONTO A CONDONAR'] = df_asignacion_test['DEUDA CAPITAL'] * df_asignacion_test['% CONDONACIÓN']
df_asignacion_test['TIPO DE FONDO'] = df_asignacion_test['TIPO DE FONDO'].fillna('NULL')

df_asignacion_test.sort_values(by=['% CONDONACIÓN'], inplace=True)
df_asignacion_test.head(5)

,CODIGO CENTRAL,N° CONTRATO,CARTERA,NOMBRE DEL CLIENTE,MONEDA,DEUDA CAPITAL,% CONDONACIÓN,GESTOR,MONTO A PAGAR,TIPO CARTERA,AGENCIA DE COBRANZA,TIPO DE FONDO,OFICINA,TERRITORIO,ID_VENTA,MONTO A CONDONAR
240732,31685884,001101559600157710,BC,JHON ABELCARPIOCISNEROS,PEN,100000.00,0.0,Cynthia Rosales,100000.00,UNSECURED,CLASA MORA,NULL,OF.NICOLAS AYLLON,DIVISION LIMA ESTE,-,0.0
212770,30041072,001101059600188494,BC,JAIRO RONALD BUENDIA LAURA,PEN,32605.95,0.0,Cynthia Rosales,32605.95,UNSECURED,ASESCOM RJ,NULL,OF.RIMAC,DIVISION LIMA CENTRO,-,0.0
212772,30041076,001100059600048389,BC,EDINSON MANUEL MARTINEZ BALCAZAR,PEN,27314.23,0.0,Cynthia Rosales,27314.23,UNSECURED,ASESCOM RJ,NULL,OFICINA MEGAPLAZA CHIMBOTE,DIVISION NORTE,-,0.0
212773,30041176,001102379600295728,EXJ,LORENA SANTIAGO MEDRANO,PEN,9980.86,0.0,Richard Gonzales Chavez,9980.86,UNSECURED,CLASA MORA,NULL,OF.EL TAMBO,DIVISION CENTRO ORIENTE,-,0.0
21846,21573431,001102875001052474,EXJ,CARLOS ENRIQUE BURGA PASTOR,PEN,978.02,0.0,Richard Gonzales Chavez,978.02,UNSECURED,CLASA MORA,NULL,OF.MCDO.MODELO CHICLAYO,DIVISION NORTE,-,0.0


In [36]:
df_asignacion_test['AGENCIA DE COBRANZA'].value_counts()

AGENCIA DE COBRANZA
ASESCOM RJ            78452
MORNESE MORA          78259
CLASA MORA            58368
Muñiz                    39
RodriguezAngobaldo       31
Name: count, dtype: int64

In [37]:
df_asignacion_test['CONTACTO BBVA'] = 'recuperaciones@opplus.bbva.com'
df_asignacion_test['CORREO BBVAA'] = 'recuperaciones@opplus.bbva.com'
df_asignacion_test['TELÉFONO'] = '595-0030'

In [39]:
correos = {
    ('ASESCOM RJ', 'BC'): 'karol.lopez@rjabogados.com',
    ('ASESCOM RJ', 'EXJ'): 'lilia.gonzales@rjabogados.com',
    ('ASESCOM RJ', 'KSTBC'): 'rosa.hoyos@rjabogados.com',
    ('CLASA MORA', 'BC'): 'rcamacho@clasaperu.com',
    ('CLASA MORA', 'EXJ'): 'jayra@clasaperu.com',
    ('CLASA MORA', 'KSTBC'): 'jsolis@clasaperu.com',
    ('MORNESE MORA', 'BC'): 'brenda.paz@mornese.pe, juan.mendoza@mornese.pe, informes@mornese.pe',
    ('MORNESE MORA', 'EXJ'): 'aparco@mornese.pe, juan.mendoza@mornese.pe, informes@mornese.pe',
    ('MORNESE MORA', 'KSTBC'): 'juan.mendoza@mornese.pe, geraldine.hinojosa@mornese.pe, informes@mornese.pe',
    ('Muñiz', None): 'szegarra@munizlaw.com',
    ('RodriguezAngobaldo', None): 'ysifuentes@er.com.pe'
}

telefonos = {
    ('ASESCOM RJ', 'BC'): '955013048, 932811608',
    ('ASESCOM RJ', 'EXJ'): '932 812 578, 932 811 752',
    ('ASESCOM RJ', 'KSTBC'): '979270273, 932810247, 945309798',
    ('CLASA MORA', 'BC'): '987414713, 977746384',
    ('CLASA MORA', 'EXJ'): '940619571, 977746384',
    ('CLASA MORA', 'KSTBC'): '936994661, 977746384',
    ('MORNESE MORA', 'BC'): '01-7057893, 977288425',
    ('MORNESE MORA', 'EXJ'): '01-7057893, 977288425',
    ('MORNESE MORA', 'KSTBC'): '01-7057893, 970168713',
    ('Muñiz', None): '999868827',
    ('RodriguezAngobaldo', None): '924104114'
}

def asignar_correo(row):
    return correos.get((row['AGENCIA DE COBRANZA'], row['CARTERA']), correos.get((row['AGENCIA DE COBRANZA'], None), None))

def asignar_telefono(row):
    return telefonos.get((row['AGENCIA DE COBRANZA'], row['CARTERA']), telefonos.get((row['AGENCIA DE COBRANZA'], None), None))

df_asignacion_test['CORREO DE LA AGENCIA'] = df_asignacion_test.apply(asignar_correo, axis=1)
df_asignacion_test['TELÉFONO DE AGENCIA'] = df_asignacion_test.apply(asignar_telefono, axis=1)

In [40]:
cols_final = ['CODIGO CENTRAL', 'N° CONTRATO', 'CARTERA', 'NOMBRE DEL CLIENTE', 'MONEDA', 'DEUDA CAPITAL', '% CONDONACIÓN', 'MONTO A CONDONAR', 'MONTO A PAGAR', 'AGENCIA DE COBRANZA', 'TIPO DE FONDO', 'TELÉFONO DE AGENCIA', 'CORREO DE LA AGENCIA', 'CONTACTO BBVA', 'CORREO BBVAA' , 'TELÉFONO', 'OFICINA', 'TERRITORIO']
df_asignacion_test = df_asignacion_test[cols_final]
print(df_asignacion_test.shape)
df_asignacion_test.head(5)

(215149, 18)


,CODIGO CENTRAL,N° CONTRATO,CARTERA,NOMBRE DEL CLIENTE,MONEDA,DEUDA CAPITAL,% CONDONACIÓN,MONTO A CONDONAR,MONTO A PAGAR,AGENCIA DE COBRANZA,TIPO DE FONDO,TELÉFONO DE AGENCIA,CORREO DE LA AGENCIA,CONTACTO BBVA,CORREO BBVAA,TELÉFONO,OFICINA,TERRITORIO
240732,31685884,001101559600157710,BC,JHON ABELCARPIOCISNEROS,PEN,100000.00,0.0,0.0,100000.00,CLASA MORA,NULL,"987414713, 977746384",rcamacho@clasaperu.com,recuperaciones@opplus.bbva.com,recuperaciones@opplus.bbva.com,595-0030,OF.NICOLAS AYLLON,DIVISION LIMA ESTE
212770,30041072,001101059600188494,BC,JAIRO RONALD BUENDIA LAURA,PEN,32605.95,0.0,0.0,32605.95,ASESCOM RJ,NULL,"955013048, 932811608",karol.lopez@rjabogados.com,recuperaciones@opplus.bbva.com,recuperaciones@opplus.bbva.com,595-0030,OF.RIMAC,DIVISION LIMA CENTRO
212772,30041076,001100059600048389,BC,EDINSON MANUEL MARTINEZ BALCAZAR,PEN,27314.23,0.0,0.0,27314.23,ASESCOM RJ,NULL,"955013048, 932811608",karol.lopez@rjabogados.com,recuperaciones@opplus.bbva.com,recuperaciones@opplus.bbva.com,595-0030,OFICINA MEGAPLAZA CHIMBOTE,DIVISION NORTE
212773,30041176,001102379600295728,EXJ,LORENA SANTIAGO MEDRANO,PEN,9980.86,0.0,0.0,9980.86,CLASA MORA,NULL,"940619571, 977746384",jayra@clasaperu.com,recuperaciones@opplus.bbva.com,recuperaciones@opplus.bbva.com,595-0030,OF.EL TAMBO,DIVISION CENTRO ORIENTE
21846,21573431,001102875001052474,EXJ,CARLOS ENRIQUE BURGA PASTOR,PEN,978.02,0.0,0.0,978.02,CLASA MORA,NULL,"940619571, 977746384",jayra@clasaperu.com,recuperaciones@opplus.bbva.com,recuperaciones@opplus.bbva.com,595-0030,OF.MCDO.MODELO CHICLAYO,DIVISION NORTE


In [30]:
df_asignacion_test.to_excel('bases/CAMPAÑAS SEPTIEMBRE 2024.xlsx', index=False)